In [1]:

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [10]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [7]:
os.listdir()

['.anaconda',
 '.astropy',
 '.bash_history',
 '.bluefish',
 '.cisco',
 '.conda',
 '.condarc',
 '.config',
 '.eclipse',
 '.editix',
 '.glue',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.matplotlib',
 '.octave_hist',
 '.oracle_jre_usage',
 '.p2',
 '.pencil',
 '.PyCharmCE2017.1',
 '.sapvi',
 '.spyder-py3',
 '.sysdb',
 '.tooling',
 '1_notMNIST.ipynb',
 '2_fullyconnected.ipynb',
 '3_regularization.ipynb',
 'AnacondaProjects',
 'AppData',
 'Application Data',
 'array.npz',
 'Contacts',
 'Cookies',
 'Desktop',
 'Documents',
 'Downloads',
 'eclipse',
 'EurekaLog',
 'export.csv',
 'export.sql',
 'Favorites',
 'HousePrices_LinearModel.ipynb',
 'IBM',
 'Links',
 'Local Settings',
 'Music',
 'My Documents',
 'my_array.npy',
 'NetHood',
 'NTUSER.DAT',
 'ntuser.dat.LOG1',
 'ntuser.dat.LOG2',
 'NTUSER.DAT{485596c3-7ed5-11e5-80df-e41d2d718e10}.TxR.0.regtrans-ms',
 'NTUSER.DAT{485596c3-7ed5-11e5-80df-e41d2d718e10}.TxR.1.regtrans-ms',
 'NTUSER.DAT{485596c3-7ed5-11e5-80df-e41d2d718e10}.TxR.2.regt

In [8]:
import os
os.listdir()

['.anaconda',
 '.astropy',
 '.bash_history',
 '.bluefish',
 '.cisco',
 '.conda',
 '.condarc',
 '.config',
 '.eclipse',
 '.editix',
 '.glue',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.matplotlib',
 '.octave_hist',
 '.oracle_jre_usage',
 '.p2',
 '.pencil',
 '.PyCharmCE2017.1',
 '.sapvi',
 '.spyder-py3',
 '.sysdb',
 '.tooling',
 '1_notMNIST.ipynb',
 '2_fullyconnected.ipynb',
 '3_regularization.ipynb',
 'AnacondaProjects',
 'AppData',
 'Application Data',
 'array.npz',
 'Contacts',
 'Cookies',
 'Desktop',
 'Documents',
 'Downloads',
 'eclipse',
 'EurekaLog',
 'export.csv',
 'export.sql',
 'Favorites',
 'HousePrices_LinearModel.ipynb',
 'IBM',
 'Links',
 'Local Settings',
 'Music',
 'My Documents',
 'my_array.npy',
 'NetHood',
 'NTUSER.DAT',
 'ntuser.dat.LOG1',
 'ntuser.dat.LOG2',
 'NTUSER.DAT{485596c3-7ed5-11e5-80df-e41d2d718e10}.TxR.0.regtrans-ms',
 'NTUSER.DAT{485596c3-7ed5-11e5-80df-e41d2d718e10}.TxR.1.regtrans-ms',
 'NTUSER.DAT{485596c3-7ed5-11e5-80df-e41d2d718e10}.TxR.2.regt

In [9]:
os.chdir('E:\Manish\Machine Learning\Deep Learning  - Udacity\Python Assignments')
os.listdir()

['1_notMNIST.ipynb',
 '2_fullyconnected.ipynb',
 '3_regularization.ipynb',
 'notMNIST.pickle',
 'notMNIST_clean.pickle',
 'notMNIST_large',
 'notMNIST_large.tar.gz',
 'notMNIST_small',
 'notMNIST_small.tar.gz']

In [11]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [12]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
num_steps = 1201

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.127371
Training accuracy: 11.9%
Validation accuracy: 13.8%
Loss at step 100: 2.334025
Training accuracy: 71.3%
Validation accuracy: 70.7%
Loss at step 200: 1.876015
Training accuracy: 74.4%
Validation accuracy: 73.0%
Loss at step 300: 1.633902
Training accuracy: 75.5%
Validation accuracy: 74.0%
Loss at step 400: 1.470576
Training accuracy: 76.4%
Validation accuracy: 74.3%
Loss at step 500: 1.348429
Training accuracy: 77.0%
Validation accuracy: 74.8%
Loss at step 600: 1.251838
Training accuracy: 77.6%
Validation accuracy: 75.0%
Loss at step 700: 1.172816
Training accuracy: 78.0%
Validation accuracy: 75.1%
Loss at step 800: 1.106623
Training accuracy: 78.5%
Validation accuracy: 75.3%
Loss at step 900: 1.050166
Training accuracy: 78.9%
Validation accuracy: 75.5%
Loss at step 1000: 1.001306
Training accuracy: 79.3%
Validation accuracy: 75.7%
Loss at step 1100: 0.958524
Training accuracy: 79.7%
Validation accuracy: 75.8%
Loss at step 1200: 0.920713
Training ac

In [14]:
batch_size = 500

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [15]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.030128
Minibatch accuracy: 12.8%
Validation accuracy: 13.0%
Minibatch loss at step 500: 1.728011
Minibatch accuracy: 72.2%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.498868
Minibatch accuracy: 71.4%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 1.120768
Minibatch accuracy: 77.4%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 0.963230
Minibatch accuracy: 76.8%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 0.867014
Minibatch accuracy: 79.2%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 0.812664
Minibatch accuracy: 80.4%
Validation accuracy: 79.5%
Minibatch loss at step 3500: 0.852609
Minibatch accuracy: 78.6%
Validation accuracy: 79.7%
Minibatch loss at step 4000: 0.902891
Minibatch accuracy: 78.6%
Validation accuracy: 80.2%
Minibatch loss at step 4500: 0.822186
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%
Minibatch loss at step 5000: 0.831091
Minibatch accuracy: 79.8%
Validation accura

In [16]:
#Changes for Relu - 1 hidden layer
batch_size = 500
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
   # Variables - intermediate
  weights_int = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_int = tf.Variable(tf.ones([hidden_nodes])/100)
    
   # Training computation - intermediate.
  logits_int = tf.matmul(tf_train_dataset, weights_int) + biases_int
  
  #print('Training set', tf_train_dataset.shape)
  #print('Logit In', logits_int.shape)  

    # Adding ReLu 
  int_output = tf.nn.relu(logits_int) 
  #print('ReLu output set', int_output.shape)  
   
    # Variables.
  weights = tf.Variable(
    tf.truncated_normal([hidden_nodes, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
    
  # Training computation.
  logits = tf.matmul(int_output, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul((tf.nn.relu(tf.matmul(tf_valid_dataset,weights_int) + biases_int)), weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul((tf.nn.relu(tf.matmul(tf_test_dataset,weights_int) + biases_int)), weights) + biases)

In [17]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 349.782013
Minibatch accuracy: 10.0%
Validation accuracy: 26.2%
Minibatch loss at step 500: 15.121041
Minibatch accuracy: 80.6%
Validation accuracy: 82.8%
Minibatch loss at step 1000: 8.821404
Minibatch accuracy: 82.4%
Validation accuracy: 83.9%
Minibatch loss at step 1500: 5.813066
Minibatch accuracy: 86.2%
Validation accuracy: 84.9%
Minibatch loss at step 2000: 5.344639
Minibatch accuracy: 84.2%
Validation accuracy: 85.5%
Minibatch loss at step 2500: 3.587094
Minibatch accuracy: 87.8%
Validation accuracy: 85.4%
Minibatch loss at step 3000: 2.454435
Minibatch accuracy: 89.4%
Validation accuracy: 86.0%
Minibatch loss at step 3500: 2.166384
Minibatch accuracy: 87.6%
Validation accuracy: 84.3%
Minibatch loss at step 4000: 3.748881
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 4500: 3.588781
Minibatch accuracy: 85.4%
Validation accuracy: 79.1%
Minibatch loss at step 5000: 1.577965
Minibatch accuracy: 87.2%
Validation accu